In [1]:
print(2)

2


In [2]:
%pwd

'c:\\Users\\thish\\OneDrive\\Pictures\\Documents\\LLM\\Medical_Chatbot\\End-to-End-Medical-Chatbot-GenerativeAI\\research'

In [3]:
import os
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\thish\\OneDrive\\Pictures\\Documents\\LLM\\Medical_Chatbot\\End-to-End-Medical-Chatbot-GenerativeAI'

In [5]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
def load_pdf_files(data):
    loader = DirectoryLoader(data, glob="**/*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [7]:
extracted_data= load_pdf_files(data='Data/')

In [8]:
#split the documents into smaller chunks
def split_documents(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    return text_splitter.split_documents(documents)



In [9]:
text_chunks = split_documents(extracted_data)
print(f"Total number of text chunks: {len(text_chunks)}")

Total number of text chunks: 5859


In [10]:
#converting to the embeddings
# !pip install sentence-transformers

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings
def download_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
embedding= download_embeddings()

C:\Users\thish\AppData\Local\Temp\ipykernel_8312\2833984598.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\thish\miniconda3\envs\MediBot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
query_res=embedding.embed_query("Hello world")
print(f"Embedding for the query 'Hello-world': ",len(query_res))  # Print first 5 dimensions of the embedding

Embedding for the query 'Hello-world':  384


In [55]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [56]:
PINECONE_API_KEY = os.getenv("PINECONE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_KEY")

In [23]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from pinecone import Pinecone, ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '2bf23de6adcff5801c63de4331218377', 'date': 'Sun, 15 Jun 2025 07:12:25 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [57]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY

In [25]:
#embed the each text chunks to vectors strore in pinecone
from langchain_pinecone import PineconeVectorStore
docsearch= PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embedding,
    index_name=index_name
)

In [26]:
#load the existing index
from langchain_pinecone import PineconeVectorStore
docsearch= PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [41]:
retriever_search=docsearch.as_retriever(search_type='similarity',search_kwargs={"k":3})

In [42]:
results=retriever_search.invoke("What is the treatment for diabetes?")
results

[Document(id='ac120bc7-e71d-4da0-b731-b55c7f40ffc8', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 274.0, 'page_label': '275', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='with a physician or pharmacist before combining tri-\ncyclic antidepressants with any other prescription or non-\nprescription (over-the-counter) medicine.\nNancy Ross-Flanigan\nAntidiabetic drugs\nDefinition\nAntidiabetic drugs are medicines that help control\nblood sugar levels in people with diabetes mellitus\n(sugar diabetes).\nPurpose\nDiabetes may be divided into type I and type II, for-\nmerly termed juvenile onset or insulin-dependent, and\nGALE ENCYCLOPEDIA OF MEDICINE 2 261\nAntidiabetic drugs'),
 Document(id='ca96d4bb-bab2-4b2d-a90a-4101a19063da', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'pa

In [60]:
# from langchain_openai import OpenAI
# llm= OpenAI(temperature=0.4,max_tokens=500)
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",  # or "gemini-1.5-pro" if supported
    temperature=0.4,
    max_output_tokens=500,
    google_api_key=GEMINI_API_KEY

)


In [61]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    "You are a helpful medical assistant. "
    "You will be provided with some medical documents. "
    "Use the information in these documents to answer the user's questions."
    "if you don't know the answer, just say 'I don't know'. use three sentences maximum to answer concise. "
    "\n\n"
    "{context}"
)

prompt= ChatPromptTemplate.from_messages(
    [
    ("system", system_prompt),
    ("human", "{input}"),
    ]
)

In [62]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever_search,question_answer_chain)


In [66]:
response=rag_chain.invoke({"input":"What is the treatment for parkisons disease?"})

In [67]:
response["answer"]

"I am sorry, but this document does not contain information about the treatment for Parkinson's disease."